In [1]:
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [56]:
import tweepy

In [57]:
api_key = "Zf5TkFjjbqxyzC02cFNbrfd2V"
api_key_secret = "afUdvsB5rueFfZhAFguKlpbhzyFwL1rEp898MSj1HBBh7In8Sk"

access_token = '1420374454392156165-XUOMDIw2YmAIcF1xZVyp4Y50JEGUlB'
access_token_secret = 'rVOG17Kl7vZf8wB3cLt9X6YRk3LrEJctL48KyqX2SPqJI'

In [58]:
keyword = 'NASDAQ stocks'

In [59]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

In [60]:
api = tweepy.API(auth)

In [61]:
public_tweets = api.search_tweets(q=keyword, tweet_mode='extended', count=1000)

In [62]:
columns = ['Time', 'User', 'Tweet']
data = []

In [63]:
for tweet in public_tweets:
    data.append([tweet.created_at, tweet.user.screen_name, tweet.full_text])

In [64]:
df = pd.DataFrame(data, columns=columns)
df.to_csv('tweets.csv')

# NLP

In [2]:
english_stopwords = stopwords.words('english')

In [3]:
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in english_stopwords]

In [4]:
stemmer = SnowballStemmer(language='english')

In [5]:
def tokenize(text):
    return [stemmer.stem(word) for word in word_tokenize(text)]

In [6]:
small_vect = CountVectorizer()
vectorizer = CountVectorizer(lowercase=True, 
                             tokenizer=tokenize,
                             stop_words=english_stopwords,
                             max_features=1000)

In [7]:
df = pd.read_csv('tweets.csv')

In [9]:
!pip install transformers
from transformers import pipeline
senti = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.9 MB/s eta 0:00:00


In [10]:
df['Result'] = df['Tweet'].apply(lambda x: senti(x))
df['sentiment'] = df['Result'].apply(lambda x: (x[0]['label']))
df['score'] = df['Result'].apply(lambda x: (x[0]['score']))

In [11]:
df['sentiment'].value_counts(normalize=True)

POSITIVE    0.94
NEGATIVE    0.06
Name: sentiment, dtype: float64

In [13]:
import plotly.express as px
fig = px.histogram(df, x='sentiment',color='sentiment',text_auto=True)
fig.show()

In [15]:
df.to_csv('tweets_sentiments.csv')

In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
MAX_ITER = 1000
model = LogisticRegression(max_iter=MAX_ITER, solver='sag')

In [18]:
train_df = pd.read_csv('tweets_sentiments.csv')

In [45]:
train_inputs = train_df['Tweet']
train_targets = train_df['sentiment']

In [47]:
# pd.DataFrame(train_inputs)
train_inputs

0     RT @PrimeXBT: 📉 The #Nasdaq is experiencing a ...
1     RT @MartyChargin: $Nasdaq The Nasdaq is up mor...
2     RT @PrimeXBT: 📉 The #Nasdaq is experiencing a ...
3     $PBH  new alert at https://t.co/A7qrDarJHY  #s...
4     $PBH  new alert at https://t.co/A7qrDarJHY  #s...
                            ...                        
95    $NE  new alert at https://t.co/A7qrDarJHY  #st...
96    #NASDAQ E-mini #Futures #NQ_F ⬆️20.75 Points (...
97    $WNC  new alert at https://t.co/A7qrDarJHY  #s...
98    $WNC  new alert at https://t.co/A7qrDarJHY  #s...
99    $ACRS  new alert at https://t.co/A7qrDarJHY  #...
Name: Tweet, Length: 100, dtype: object

In [49]:
%%time
vectorizer.fit(train_inputs)
# vectorizer.get_feature_names_out()
train_inputs = vectorizer.transform(train_inputs)

CPU times: user 121 ms, sys: 2.69 ms, total: 124 ms
Wall time: 246 ms


In [50]:
%%time
model.fit(train_inputs, train_targets)

CPU times: user 18.1 ms, sys: 941 µs, total: 19 ms
Wall time: 22.4 ms


LogisticRegression(max_iter=1000, solver='sag')

In [51]:
train_preds = model.predict(train_inputs)
train_targets

0     POSITIVE
1     POSITIVE
2     POSITIVE
3     POSITIVE
4     POSITIVE
        ...   
95    POSITIVE
96    POSITIVE
97    POSITIVE
98    POSITIVE
99    POSITIVE
Name: sentiment, Length: 100, dtype: object

In [52]:
pd.Series(train_preds).value_counts()

POSITIVE    96
NEGATIVE     4
dtype: int64

In [53]:
pd.Series(train_targets).value_counts()

POSITIVE    94
NEGATIVE     6
Name: sentiment, dtype: int64

In [54]:
from sklearn.metrics import accuracy_score

In [55]:
accuracy_score(train_targets, train_preds)

0.98